## Dependency Inversion Principle (DIP)

The Dependency Inversion Principle is the D in SOLID. It’s a design rule that reduces coupling and increases architectural stability.

### 📌 The Principle (canonical form)
According to the definition:
- High‑level modules should not depend on low‑level modules. Both should depend on abstractions.
- Abstractions should not depend on details. Details should depend on abstractions.
This “inverts” the usual direction of dependency: instead of high‑level code reaching down into concrete implementations, both layers meet at an interface.

### 🎯 Why DIP matters (operationally)
DIP prevents:
- Fragile coupling — changes in low‑level code breaking high‑level logic.
- Rigid architectures — adding new implementations requires modifying high‑level modules.
- Leaky abstractions — high‑level modules knowing too much about implementation details.
DIP is basically the architectural guardrail that ensures your domain layer never leaks into infrastructure.

## references
- https://en.wikipedia.org/wiki/Dependency_inversion_principle

In [1]:
# no dip
# Problems:
# - You cannot swap logging strategies without modifying OrderService.
# - High-level policy code knows about low-level details.

class FileLogger:
    def log(self, msg):
        print(f"[FILE] {msg}")

class OrderService:
    def __init__(self):
        self.logger = FileLogger()  # hard dependency

    def place_order(self):
        self.logger.log("Order placed")

In [2]:
# with dip
# - High-level (OrderService) depends only on Logger (an abstraction).
# - Low-level (FileLogger) also depends on Logger.
# - You can inject any logger (DB, cloud, null, test double).
# This is exactly the “inversion” described in the sources: low-level details depend on abstractions, not the other way around.


from abc import ABC, abstractmethod

class Logger(ABC):
    @abstractmethod
    def log(self, msg): ...

class FileLogger(Logger):
    def log(self, msg):
        print(f"[FILE] {msg}")

class OrderService:
    def __init__(self, logger: Logger):
        self.logger = logger

    def place_order(self):
        self.logger.log("Order placed")

### Applying DIP in Python
- Define protocols/ABCs for collaborators.
- Inject dependencies via constructor parameters (or provider/factory).
- Keep composition at the boundary (main/bootstrap), not deep inside logic.
- Prefer small interfaces focused on capabilities, not big god objects.

In [ ]:
from typing import Protocol, Iterable

class OrderRepo(Protocol):
    def save(self, order: dict) -> None: ...
    def all(self) -> Iterable[dict]: ...

class InMemoryOrderRepo:
    def __init__(self):
        self._orders = []
    def save(self, order):
        self._orders.append(order)
    def all(self):
        return list(self._orders)

class OrderService:
    def __init__(self, repo: OrderRepo, logger):
        self.repo = repo
        self.logger = logger

    def create(self, order: dict):
        self.repo.save(order)
        self.logger.log(f"Saved {order}")
        return order

class NullLogger:
    def log(self, msg):
        pass

repo = InMemoryOrderRepo()
svc = OrderService(repo, NullLogger())
svc.create({"id": 1, "item": "book"})
repo.all()

Try swapping implementations:
- Replace `InMemoryOrderRepo` with a fake that counts calls for tests.
- Replace `NullLogger` with a logger that raises on certain messages to assert flows.